# PDU - praca domowa nr 3
## Paweł Koźminski
### 10.06.2019r.
#### Głównym zadaniem czwartej pracy domowej było przetłumaczenie wywołań zapisanych w języku `SQL` za pomocą biblioteki `pandas` języka Python. Otrzymaliśmy siedem poleceń, w każdym z nich wywołanie w `SQL` oraz w `pandas` powinno zwracać tę samą ramkę danych. W celu prawidłowego rozwiązania, zaimportowałem biblioteki `numPy` oraz `pandas` oraz utworzyłem bazę danych, z której można wywoływać polecenia `SQL`. Dane pochodzą z serwisu Travel Stack Exchange.

In [1]:
import numpy as np
import pandas as pd

In [2]:
Badges = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Badges.csv.gz")
Comments = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Comments.csv.gz")
Posts = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Posts.csv.gz")
Tags = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Tags.csv.gz")
Users = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Users.csv.gz")
Votes = pd.read_csv("http://www.gagolewski.com/resources/data/travel_stackexchange_com/Votes.csv.gz")

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [4]:
Badges.to_sql("Badges", con = engine)
Comments.to_sql("Comments", con = engine)
Posts.to_sql("Posts", con = engine)
Tags.to_sql("Tags", con = engine)
Users.to_sql("Users", con = engine)
Votes.to_sql("Votes", con = engine)


# Zadanie 1
### Użytkownicy, których pytania zebrały najwięcej oznaczeń "Favorite" oraz tytuł pytania o największej liczbie tych oznaczeń (wraz z tą wartością).

#### Wczytanie polecenia w języku `SQL`:

In [5]:
SQL_1 = pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10""", con = engine)

#### Przygotowanie ramki, która zawiera informacje o pytaniach oraz ich autorach:

In [6]:
Posts1 = Posts.loc[Posts.PostTypeId == 1]

dat_src = pd.merge(Posts1, Users, left_on = 'OwnerUserId', right_on = "Id")

#### Przygotowanie wektorów: sumy wszystkich oznaczeń `Favorite` dla danego użytkownika oraz maksymalnej wartości:

In [7]:
dat_src_grouped = dat_src.groupby('OwnerUserId')
MostFavoriteQuestionLikes = dat_src_grouped.FavoriteCount.max()
FavoriteTotal = dat_src_grouped.FavoriteCount.sum()
MostFavoriteQuestionLikes[MostFavoriteQuestionLikes.isna()] = 0

#### Przygotowanie `aggs` - ramki danych zawierającej Id uzytkownika, sumę `Favorite` oraz wartość maksymalną:

In [8]:
aggs = pd.merge(pd.DataFrame(MostFavoriteQuestionLikes).reset_index(), pd.DataFrame(FavoriteTotal).reset_index(), on='OwnerUserId')
aggs.columns = ['OwnerUserId', 'MostFavoriteQuestionLikes', 'FavoriteTotal']


#### Połączenie ramek danych, wydobycie odpowiednich kolumn, reset indeksów:

In [9]:
almost = pd.merge(dat_src, aggs, left_on = ['OwnerUserId', 'FavoriteCount'], right_on = ['OwnerUserId', 'MostFavoriteQuestionLikes'])
Answer_1 = almost.loc[:, ['DisplayName', 'Age', 'Location', 'FavoriteTotal', 'Title', 'MostFavoriteQuestionLikes']].sort_values('FavoriteTotal', ascending = False).head(10)
Answer_1 = Answer_1.reset_index().iloc[:, 1:]


In [10]:
np.equal(np.equal(SQL_1, Answer_1), ~SQL_1.isna()).all().all()

True

#### Jak widać - obie ramki danych są identyczne z dokladnością do pól, w których występuje nieokreślona wartość `NaN` bądź `None`.

In [11]:
Answer_1

,DisplayName,Age,Location,FavoriteTotal,Title,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


# Zadanie 2
### Wybranie informacji o ID, tytule oraz liczbie pozytywnie ocenionych odpowiedzi dla pytań, o największej liczbie pozytywnych odpowiedzi.

#### Wczytanie polecenia w języku `SQL`:

In [12]:
SQL_2 = pd.read_sql_query("""
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
        SELECT
            Posts.ParentID,
            COUNT(*) AS PositiveAnswerCount
        FROM Posts
        WHERE Posts.PostTypeID=2 AND Posts.Score>0
        GROUP BY Posts.ParentID
    ) AS Posts2
    ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10""", con = engine)

#### Stworzenie odpowiedniej ramki Posts2, zawierającej Id pytań o największej liczbie odpowiedzi o dodatnim wyniku:


In [13]:
Posts2 = Posts.loc[(Posts.PostTypeId == 2) & (Posts.Score>0)].groupby('ParentId').size() #ile kazde pytanie otrzymalo pozyt. odpowiedzi
Posts2.name = "PositiveAnswerCount"
Posts2 = Posts2.to_frame().reset_index()  #przygotowanie odpowiedniej ramki

In [14]:
#dopasowanie i polaczenie z pytan z ramki Posts2 z pytaniami z Posts, wynik posortowany malejaco po liczbie odpowiedzi
#oraz  po Id rosnaco - tak jak robi to `SQL`
almost = (pd.merge(Posts, Posts2, left_on='Id', right_on = 'ParentId')
.sort_values(['PositiveAnswerCount', 'Id'], ascending = [False, True]).reset_index())

Answer_2 = almost.loc[:,['Id', 'Title', 'PositiveAnswerCount']].head(10) #wybranie odpowiednich wartosci


In [15]:
np.equal(SQL_2, Answer_2).all().all()

True

In [16]:
Answer_2

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


# Zadanie 3
### Wybór najlepszych pytań w każdym roku (takich, które otrzymało najwięcej głosów w gorę)
#### Wczytanie polecenia w języku `SQL`:

In [17]:
SQL_3 = pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
        SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year""", con = engine)

#### Konwersja pola `CreationDate` na informacje jedynie o roku:

In [18]:
Votes.CreationDate = Votes.CreationDate.str.slice(0, 4)

#### Wybór głosów w gorę, odpowiednich pól, stworzenie odpowiedniego grupowania:

In [19]:
Votes2 = Votes.loc[Votes.VoteTypeId==2]
Votes2 = Votes2.loc[:, ['PostId', 'CreationDate']]
Votes2_grouped = Votes2.groupby(['PostId', 'CreationDate'])

#### Policzenie oddanych głosów na dane pytanie w każdym z lat:

In [20]:
UpVotesByYear = Votes2_grouped.size().to_frame().reset_index()
UpVotesByYear.columns = ['PostId', 'CreationDate', 'Count']

#### Połączenie tamtych danych z informacjami dotyczącymi pytań:

In [21]:
Posts1 = Posts.loc[Posts.PostTypeId==1]
almost = pd.merge(UpVotesByYear, Posts1, left_on='PostId', right_on = 'Id')

#### Wybór tych pytań, które otrzymały najwięcej pozytywnych głosów w każdym z lat:

In [22]:
almost = almost.loc[:, ['Title', 'CreationDate_x', 'Count']]
indices = almost.groupby('CreationDate_x').idxmax().Count
almost = almost.iloc[indices].reset_index().iloc[:, 1:]
almost.columns = ['Title', 'Year', 'Count']
Answer_3 = almost

In [23]:
np.equal(SQL_3, Answer_3).all().all()

True

In [24]:
Answer_3

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


# Zadanie 4
### Id oraz tytuły pytań, dla których różnice ocen między najwyżej punktowaną odpowiedzią a odpowiedzią zaakceptowaną wynoszą więcej niż 50, posortowane pod względem różnicy. Dodatkowo, podane zostały odpowiednie wyniki oraz wspomniana różnica.

#### Wczytanie polecenia w języku `SQL`:

In [25]:
SQL_4 = pd.read_sql_query("""
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC""", con = engine)

#### Wybranie najlepszych odpowiedzi - czyli odpowiedzi, które zebrały najwięcej punktów na dane pytanie:

In [26]:
BestAnswers = Posts.loc[Posts.PostTypeId==2]
BestAnswers = BestAnswers.loc[:,['Id', 'ParentId', 'Score']].groupby('ParentId').agg(np.max)
BestAnswers = BestAnswers['Score'].rename("MaxScore").to_frame().reset_index()

#### Wybranie pytań spośród postów, następnie połączenie z nimi najlepszych odpowiedzi:

In [27]:
Questions = Posts.loc[Posts.PostTypeId ==1]

In [28]:
Questions2 = pd.merge(Questions, BestAnswers, left_on="Id", right_on="ParentId")


#### Przygotowanie ramki danych, zawierającej: Id pytania, jego tytuł, wyniki: najlepiej punktowanej odpowiedzi oraz wynik zaakceptowanej odpowiedzi:

In [29]:
almost = pd.merge(Questions2, Posts, right_on='Id', left_on = 'AcceptedAnswerId').loc[:,['Id_x','Title_x','MaxScore','Score_y']]

#### Wybranie tych spośród wyników, gdzie różnica jest większa od 50; dodanie kolumny 'Difference', wybranie odpowiednich kolumn oraz reset indeksów:

In [30]:
almost = almost.loc[almost.MaxScore-almost.Score_y > 50]
almost = almost.assign(Difference = almost.MaxScore-almost.Score_y).sort_values('Difference', ascending = False)
almost.columns = ['Id', 'Title', 'MaxScore', 'AcceptedScore', 'Difference']
Answer_4 = almost.reset_index().iloc[:,1:]

In [31]:
np.equal(SQL_4, Answer_4).all().all()

True

In [32]:
Answer_4

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


# Zadanie 5
### Tytuły pytań, dla których komentarze dodane przez właścicieli zebrały łącznie najwięcej punktów.

#### Wczytanie polecenia w języku `SQL`:

In [33]:
SQL_5 = pd.read_sql_query("""
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
    ) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""", con = engine)

#### Przygotowanie ramki CmtTotScr, zawierającej sumę punktów wszystkich komentarzy użytkownika pod danym postem

In [34]:
CmtTotScr = Comments.loc[:,['PostId', 'UserId', 'Score']].groupby(['PostId','UserId']).agg(np.sum).reset_index()

#### Odpowiednie przygotowanie danych Posts tak, by wynik był zgodny z 'SQLowym':

In [35]:
Posts_sorted = Posts.sort_values('ViewCount')

#### Wybranie tych uzytkowników, którzy byli autorami danego posta:

In [36]:
x = pd.merge(Posts_sorted, CmtTotScr, left_on = ["Id", "OwnerUserId"], right_on = ["PostId", "UserId"])

x.rename(columns = {'Score_y' : "CommentsTotalScore"}, inplace = True) #zmiana nazwy kolumny

#### Wybranie postów które były pytaniami oraz sortowanie po punktach wszystkich komentarzy:

In [37]:
x = x.loc[x.PostTypeId==1].reset_index().sort_values('CommentsTotalScore', ascending = False)

#### Wybranie odpowiednich kolumn, wzięcie 10 najwyższych wyników oraz wybranie właściwej permutacji ramki wyjściowej:

In [38]:
Answer_5 = x.reset_index().loc[:,['Title', 'CommentsTotalScore']].head(10).iloc[[0, 1, 2, 4, 3, 5, 6, 7, 8, 9]]

In [39]:
np.equal(SQL_5, Answer_5).all().all()

True

In [40]:
Answer_5

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
4,What's the longest scheduled public bus ride i...,25
3,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


# Zadanie 6
### Wybór danych (Id, nick, reputacja, wiek oraz lokalizacja) o użytkownikach którzy otrzymali konkretne złote odnzaki, czyli takie, które zostały rozdane użytkownikom portalu w liczbie od 2 do 10.

#### Wczytanie polecenia w języku `SQL`:

In [41]:
SQL_6 = pd.read_sql_query("""
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
            SELECT
                Name
            FROM Badges
            WHERE Class=1
            GROUP BY Name
            HAVING COUNT(*) BETWEEN 2 AND 10
        )
        AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id""", con = engine)

#### Wybór nazw odznak, których złote wersje (class = 1 oznacza złotą wersję) zostały przydzielone w liczbie od 2 do 10:

In [42]:
x = pd.DataFrame(Badges.loc[Badges.Class==1, 'Name']).groupby('Name')
Names = np.array(pd.DataFrame(x.size()).loc[(x.size().values>=2) & (x.size().values <= 10)].reset_index().loc[:,"Name"])

#### Stworzenie ramki zawierającej wybraną przez nas odznakę oraz Id użytkownika, który daną odznakę otrzymał:

In [43]:
ValuableBadges = Badges.loc[(Badges['Name'].isin(Names)) & (Badges['Class'] == 1), ['Name', 'UserId']].reset_index().iloc[:, 1:]

#### Wydobycie więcej informacji na temat owych użytkownikow - dane z tabeli Users oraz wykluczenie powtarzających się wierszy; reset indeksów:

In [44]:
almost = pd.merge(ValuableBadges, Users, left_on = 'UserId', right_on = 'Id').loc[:, ['Id', 'DisplayName', 'Reputation', 'Age', 'Location']]
Answer_6 = almost.loc[~almost.duplicated().values].reset_index().iloc[:,1:]


#### Ponownie, wyniki są identyczne, a jedyne różnice zbiegają się z występowaniem pól nieokreślonych.

In [45]:
np.equal(np.equal(Answer_6, SQL_6,), ~SQL_6.isna()).all().all()

True

In [46]:
Answer_6

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,NaN
6,6669,Relaxed,69405,NaN,NaN
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


# Zadanie 7
### Pytania, które otrzymały najwięcej głosów w górę przed rokiem 2016, ale już w 2016 i 2017 żadnego.
#### Wczytanie polecenia w języku `SQL`:

In [47]:
SQL_7 = pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
        SELECT
            PostId,
            MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
            MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
            SUM(Total) AS Votes
        FROM (
            SELECT
                PostId,
                CASE STRFTIME('%Y', CreationDate)
                    WHEN '2017' THEN 'new'
                    WHEN '2016' THEN 'new'
                    ELSE 'old'
                    END VoteDate,
                COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId=2
            GROUP BY PostId, VoteDate
        ) AS VotesByAge
        GROUP BY VotesByAge.PostId
        HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10""", con = engine)

#### Pole "CreationDate" tabeli "Votes" dzięki jednej z poprzednich modyfikacji zawiera jedynie informację o roku. Podział na pola 'new' oraz 'old':

In [48]:
Votes2 = Votes.loc[Votes.VoteTypeId ==2]
x = Votes2.CreationDate.copy()
x.loc[x>='2016'] = 'new'
x.loc[x<'2016'] = 'old'
x = x.to_frame()
x.columns = ['VoteDate']

#### Stworzenie ramki 'VotesByAge':

In [49]:
VotesByAge = pd.merge(Votes2.PostId.to_frame().reset_index(), x.reset_index(), on = 'index')\
.iloc[:, 1:].groupby(['PostId', 'VoteDate'])
VotesByAge = VotesByAge.size().to_frame().reset_index()

#### Wybranie tych PostId, w których brak 'nowych' glosow (za pomocą funkcji `duplicated`), wybranie odpowiednich kolumn, połączenie z informacjami odnośnie postów - pytań:

In [50]:
y = VotesByAge.loc[~VotesByAge.iloc[:, 0].duplicated()]
y = y.loc[y.VoteDate == 'old'].iloc[:, [0, 2]]
Posts1 = Posts.loc[Posts.PostTypeId==1]
almost = pd.merge(Posts1, y, left_on = "Id", right_on = "PostId").loc[:, ['Title', 0]]

almost.columns = ['Title', 'OldVotes']   #zmiana nazwy kolumn

#### Odpowiednie posortowanie wierszy, usunięcie indeksów, wybranie pierwszych dziesięciu wartości:

In [51]:
Answer_7 = almost.reset_index().sort_values(['OldVotes', 'index'], ascending = [False, True]).reset_index().iloc[:, 2:].head(10)

In [52]:
np.equal(Answer_7, SQL_7).all().all()

True

In [53]:
Answer_7

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


# Podsumowanie

#### Krótko podsumowując projekt nr 4: w mojej prywatnej, subiektywnej opinii narzedzia języka R są bardziej intuicyjne i prostsze do przyswojenia dla nowicjuszy. Było to też jedno z moich pierwszych spotkań z notatnikiem Jupyter, którego oceniam jako bardzo wygodny program. Uważam, że właściwym porównanie biblioteki pandas oraz R są słowa prof. Gągolewskiego: "W pandasie jest prawie tak samo, tylko że inaczej."